In [ ]:
import os
import sqlite3
import yfinance as yf

# Define the list of ticker symbols
tickers = ['MSFT', 'GOOG', 'AAPL', 'NVDA','AXISBANK.NS','BAJFINANCE.NS','BANKBARODA.NS', 'FEDRALBNK.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'IDFCFIRSTB.NS', 'INDUSINDBK.NS', 'INFY.NS', 'KOTAKBANK.NS', 'PNB.NS', 'RBLBANK.NS', 'SBIN.NS', 'TCS.NS', 'WIPRO.NS']
#tickers = ['AAPL', 'NVDA']

tickers =['TSLA']
# Define the start date
start_date = '2012-12-01'

# Create SQLite3 database if it doesn't exist
db_file = 'stock_data.db'
conn = sqlite3.connect(db_file)

# Create the Data directory if it doesn't exist
data_directory = 'Data'
if not os.path.exists(data_directory):
    os.makedirs(data_directory)

# Loop through each ticker
for ticker in tickers:
    # Fetch the data
    data = yf.download(ticker, start=start_date)

    print(f"{ticker} - {len(data)}")
    
    # Save data to CSV in the Data directory
    file_path = os.path.join(data_directory, f'{ticker}_prices.csv')
    data.to_csv(file_path)
    
    # Create a table in the database for each ticker's data
    data.to_sql(ticker.replace(".", "_"), conn, if_exists='replace')

# Close the database connection
conn.close()


In [ ]:
import os

In [ ]:
import os
import csv
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('stock_data.db')
c = conn.cursor()

# Create Prices table if not exists
c.execute('''CREATE TABLE IF NOT EXISTS Prices (
                StockName TEXT,
                Date TEXT,
                Open REAL,
                High REAL,
                Low REAL,
                Close REAL,
                AdjClose REAL,
                Volume INTEGER
             )''')

# Folder containing stock data files
folder_path = 'Data'

# Iterate through files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('TSLA_prices.csv'):
        # Extract stock name from file name
        stock_name = file_name.split('.')[0]

        # Open and read CSV file
        with open(os.path.join(folder_path, file_name), 'r') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header row
            for row in reader:
                # Insert data into Prices table
                c.execute('''INSERT INTO Prices (StockName, Date, Open, High, Low, Close, AdjClose, Volume)
                             VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                          (stock_name, row[0], float(row[1]), float(row[2]), float(row[3]),
                           float(row[4]), float(row[5]), int(row[6])))

# Commit changes and close connection
conn.commit()
conn.close()


In [ ]:
import numpy as np

def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(data, fast_period=12, slow_period=26, signal_period=9):
    exp1 = data.ewm(span=fast_period, adjust=False).mean()
    exp2 = data.ewm(span=slow_period, adjust=False).mean()
    macd_line = exp1 - exp2
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    macd_histogram = macd_line - signal_line
    return macd_line, signal_line, macd_histogram

def calculate_bollinger_bands(data, window=20, num_std=2):
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_std)
    lower_band = rolling_mean - (rolling_std * num_std)
    return upper_band, rolling_mean, lower_band


In [ ]:
conn.close()

In [10]:
import sqlite3
import pandas as pd
import numpy as np

# Define the list of ticker symbols
tickers = ['MSFT', 'GOOG', 'AAPL', 'NVDA','AXISBANK.NS','BAJFINANCE.NS','BANKBARODA.NS', 'FEDRALBNK.NS', 'HDFCBANK.NS', 'ICICIBANK.NS', 'IDFCFIRSTB.NS', 'INDUSINDBK.NS', 'INFY.NS', 'KOTAKBANK.NS', 'PNB.NS', 'RBLBANK.NS', 'SBIN.NS', 'TCS.NS', 'WIPRO.NS']
tickers = ['TSLA','MSFT', 'GOOG', 'AAPL', 'NVDA']
# Connect to SQLite database
conn = sqlite3.connect('stock_data.db')
df_new = pd.DataFrame()
# Loop through each ticker
for ticker in tickers:
    print(f"Processing ticker {ticker}")
    # Read data from SQLite table into a DataFrame
    
    df = pd.read_sql_query(f"SELECT * FROM prices where StockName='{ticker}' order by Date", conn)
    df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
    # Calculate moving averages
    df['MA5'] = df['Close'].rolling(window=5).mean()
    df['MA10'] = df['Close'].rolling(window=10).mean()
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA100'] = df['Close'].rolling(window=100).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()

    # Calculate RSI
    df['RSI'] = calculate_rsi(df['Close'])

    # Calculate Bollinger Bands
    upper, middle, lower = calculate_bollinger_bands(df['Close'])
    df['UpperBB'] = upper
    df['MiddleBB'] = middle
    df['LowerBB'] = lower

    # Calculate MACD
    macd, signal, _ = calculate_macd(df['Close'])
    df['MACD'] = macd
    df['MACD_Signal'] = signal

    df_new = pd.concat([df_new,df])

# Update SQLite table with DataFrame containing additional columns
df_new.to_sql(f'new_prices', conn, if_exists='append', index=False)

# Close the database connection
conn.close()



Processing ticker TSLA
Processing ticker MSFT
Processing ticker GOOG
Processing ticker AAPL
Processing ticker NVDA


In [ ]:
import os